Desarrollo de modelo ML en relación ítem-ítem, utilizando el algoritmo de k-NearestNeighbors. Tomando como elementos para medir la sercania de los juegos los generos de df_games y los años de lanzamiento.
No se utilizan los df_review ya que no siempre coinciden los id de los juegos, y al eliminar los valores nulos que surgen de unir los df se pierde informacion . Por esto se toma solo df_games como directorio pricipal de los juegos.

In [1]:
import pandas as pd
import numpy as np

In [3]:
from sklearn.neighbors import NearestNeighbors

In [2]:

df_games = pd.read_csv('df_games.csv')

In [26]:
df_games.tail()

,app_name,item_id,año,Accounting,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
29776,Kebab it Up!,745400,2018,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
29777,Colony On Mars,773640,2018,0,0,0,0,0,1,0,...,0,0,0,1,0,0,1,0,0,0
29778,LOGistICAL: South Africa,733530,2018,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
29779,Russian Roads,610660,2018,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
29780,EXIT 2 - Directions,658870,2017,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [117]:
columnas_generos = df_games.columns.difference(['app_name', 'item_id', 'año']).tolist()


In [118]:
# combino las columnas y elimino los nulos que se podrian haber formado por las dudas
matriz_caracteristicas_combinada = df_games[columnas_generos]

# entreno el modelo NearestNeighbors con la matriz combinada
nn_model = NearestNeighbors(metric='cosine')
nn_model.fit(matriz_caracteristicas_combinada)

NearestNeighbors(metric='cosine')

Finalmente incorporamos el modelo a la funcion de recomendacion

In [115]:
def obtener_recomendaciones(id_juego, df_games, n=5):
    # Seleccionar columnas de géneros relevantes para el entrenamiento
    columnas_generos = df_games.columns.difference(['app_name', 'item_id', 'año']).tolist()

    # Obtener matriz de características combinada solo con los géneros
    matriz_caracteristicas_combinada = df_games[columnas_generos]

    # Entrenar el modelo NearestNeighbors con la matriz de géneros
    nn_model = NearestNeighbors(metric='cosine')
    nn_model.fit(matriz_caracteristicas_combinada)

    # Buscar los vecinos más cercanos del juego dado
    if id_juego not in df_games['item_id'].values:
        return "ID de juego no encontrado en el DataFrame"

    idx = df_games[df_games['item_id'] == id_juego].index[0]  # Índice en el DataFrame

    # Obtener características del juego seleccionado
    juego_seleccionado = [matriz_caracteristicas_combinada.iloc[idx]]

    distances, indices = nn_model.kneighbors(juego_seleccionado, n_neighbors=n+1)
    indices = indices.flatten()[1:]

    juegos_recomendados = df_games.iloc[indices][['item_id', 'app_name']]
    recomendaciones = juegos_recomendados.to_dict(orient='records')

    return recomendaciones


In [119]:
obtener_recomendaciones(352760, df_games)

c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


[{'item_id': 210490, 'app_name': 'Fray: Reloaded Edition'},
 {'item_id': 449440, 'app_name': 'The Big Elk'},
 {'item_id': 453400, 'app_name': 'Squad - Original Soundtrack Vol. 1 & 2'},
 {'item_id': 368050, 'app_name': 'Almightree: The Last Dreamer'},
 {'item_id': 284580, 'app_name': 'ZAMB! Biomutant Extermination'}]